In [232]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import pandas as pd
from pandas.api.types import CategoricalDtype
import scipy as sp
from pathlib import Path
import tkinter as tk
from tkinter import filedialog
import itertools
from colour import Color
from datetime import date
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import MaxNLocator
from scipy import stats
from read_roi import read_roi_zip
from collections import OrderedDict, Counter
import math
import PIL
from PIL import Image
from PIL.TiffTags import TAGS

In [233]:
today = date.today()
print("Today's date:", today)
date = str(today)
date

Today's date: 2023-07-03


'2023-07-03'

In [234]:
#set variables

tubulin = '[Tubulin] ' r'$(\mu M)$'
tub = 'tub'

DCXconc = '[DCX] ' r'$(n M)$'
DCX = 'DCX'
Type = 'DCX Type'

EBconc = '[EB] ' r'$(n M)$'
EB = 'EB'
Expt = 'Expt'

Concentration = 'Concentration ' r'$(\mu M)$'
Length = 'Length ' r'$(\mu m)$'
Lifetime = 'Lifetime ' r'$(min)$'
GrowthRate = 'Growth Rate ' r'$(\mu m / min)$'
TimeToNucleate = 'Time to Nucleate ' r'$(min)$'
ShrinkageLength = 'Shrink Length ' r'$(\mu m)$'
ShrinkageLifetime = 'Shrink Lifetime ' r'$(min)$'
ShrinkageRate = 'Shrink Rate ' r'$(\mu m / min)$'

investigator = 'Brandi'

In [178]:
#Type_order = ['None', 'Y64N','R76S','R102S', 'R178L', 'P191R', 'G223E','R303X', 'WT']
Type_order = [#'None',  'WT', 'R76S', 
    'P191R'
    #, 'P191R-agg'
]
category_Type = CategoricalDtype(Type_order, ordered=True,) #order mutants

In [347]:
#open directory

askdirectory = filedialog.askdirectory() # show an "Open" dialog box and return the path to the selected file
path = Path(askdirectory)
path

WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/Mutant aggregation/tubulin/2023-05-16_DCX-EB')

In [348]:
#search for relevant data files

dateloc = str(path).find("202")
date = str(path)[dateloc : dateloc+10]

folders = [x for x in path.iterdir() if x.is_dir()]
folders = [i for i in folders if (str(i)[-7:] == 'finalDS') == True]

ch = []
conc_tub = []
conc_DCX = []
conc_EB = []
DCX_type = []

for i in folders:
    CHloc = str(i).rfind("Ch")
    tubloc = str(i).rfind(tub)
    DCXloc = str(i).rfind(DCX)
    dcxtypeloc = str(i).find("nM_EB")
    EBloc = str(i).rfind(EB)
    
    ch = ch + [str(i)[CHloc+2:CHloc+3]]
    conc_tub = conc_tub + [(str(i)[tubloc+3:tubloc+5])]
    conc_DCX = conc_DCX + [(str(i)[dcxtypeloc-2:dcxtypeloc])]
    conc_EB = conc_EB + [(str(i)[EBloc+2:EBloc+4])]
    DCX_type = DCX_type + [(str(i)[DCXloc+4:dcxtypeloc-3])]

date,ch, conc_tub, conc_DCX, DCX_type, conc_EB


('2023-05-16', ['1'], ['08'], ['00'], ['None'], ['00'])

In [349]:
#get values for concentration of species

tub_conc =  [float(i) for i in conc_tub]
DCX_conc =  [float(i) for i in conc_DCX]
EB_conc = [float(i) for i in conc_EB]


In [350]:
#calculate pixel scale 

frame_analyzer = pd.read_csv(path/'frame_rates_avg.csv',sep=',')
pixel =  0.107 #T2 0.107 #T1 0.0633
folders

[WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/Mutant aggregation/tubulin/2023-05-16_DCX-EB/2023-05-16_Ch1-tub08uM_DCX-None-00nM_EB00nM_finalDS')]

In [351]:
#frame_analyzer = frame_analyzer.iloc[[1,3]].reset_index()
frame_analyzer

,Directory,Mean,SD,Frames,Outliers,MeanWOutliers,SDWOutliers
0,C:\Users\Brouhard Lab\Data\Results_Files\Mutan...,5.02,0.02,400,1,19.53,289.84


In [352]:
#Build Data Frame (empty)

def open_roi_poly(k,name):
    roi_path = folders[k]/(name)

    roi_data = read_roi_zip(roi_path)
    roi_df = pd.DataFrame(columns=['Track ID','x1','x2','x3','y1','y2','y3'])
    
    for key in roi_data:
        roi_df1 = pd.DataFrame({'Track ID': [roi_data[key]['name']],'x1': [roi_data[key]['x'][0]],'x2': [roi_data[key]['x'][1]],'x3': [roi_data[key]['x'][2]],'y1': [roi_data[key]['y'][0]],'y2': [roi_data[key]['y'][1]],'y3': [roi_data[key]['y'][2]]})
        roi_df = pd.concat([roi_df,roi_df1])
    
    roi_df = roi_df.reset_index(drop = True)
    return roi_df

In [353]:
#Read ROI data

open_roi_poly(0,'kymoslopes.zip')

,Track ID,x1,x2,x3,y1,y2,y3
0,0046-0026,29.25,22.25,30.0,38.0,52.5,53.0
1,0043-0032,24.833334,41.5,23.166668,14.5,71.833328,71.5
2,0114-0012,22.0,1.5,19.5,94.5,132.0,133.0
3,0038-0009,17.0,0.0,18.0,14.0,58.5,61.0
4,0072-0061,33,89,35,10,132,134
...,...,...,...,...,...,...,...
100,0089-0071,40,102,41,28,150,150
101,0119-0071,44,98,44,61,176,177
102,0093-0089,54,125,53,18,166,167
103,0052-0055,37,72,37,19,84,85


In [354]:
#Read data for T=0 correction

roi_path = folders[0]/('start.zip')
roi_data = read_roi_zip(roi_path)
roi_data

OrderedDict([('0004-0064',
              {'type': 'polyline',
               'x': [57.875, 65.625, 69.25],
               'y': [7.375, 3.25, -0.375],
               'n': 3,
               'width': 0,
               'name': '0004-0064',
               'position': 0}),
             ('0004-0075',
              {'type': 'polyline',
               'x': [72.375, 76.25, 78.125],
               'y': [7.375, 3.625, -0.375],
               'n': 3,
               'width': 0,
               'name': '0004-0075',
               'position': 0})])

In [355]:
#Define data Parameters

Length = 'Length ' r'$(\mu m)$'
Lifetime = 'Lifetime ' r'$(min)$'
GrowthRate = 'Growth Rate ' r'$(\mu m / min)$'
TimeToNucleate = 'Time to Nucleate ' r'$(min)$'
ShrinkageLength = 'Shrink Length ' r'$(\mu m)$'
ShrinkageLifetime = 'Shrink Lifetime ' r'$(min)$'
ShrinkageRate = 'Shrink Rate ' r'$(\mu m / min)$'
poly_parameter_names = (Length,Lifetime,GrowthRate,TimeToNucleate)
line_parameter_names = (Length,Lifetime,GrowthRate,TimeToNucleate,ShrinkageLength,ShrinkageLifetime,ShrinkageRate)

In [356]:
#Extract parameter data to fill data frame

def poly_dynamics(k):
        
    data = open_roi_poly(k,'kymoslopes.zip')
    supp = open_roi_poly(k,'start.zip')
    
    spf = frame_analyzer['Mean'][k];
    frames = frame_analyzer['Frames'][k]
    
    suppn = len(supp)
    start = sum(abs(supp['y2']-supp['y1']))/suppn;
    
    n = len(data)
    pre_df ={'Date': np.full((n), date), 'Investigator': np.full((n), investigator),
              'Ch': np.full((n), ch[k]),tubulin: np.full((n), tub_conc[k]),DCXconc: np.full((n), DCX_conc[k]),EBconc: np.full((n), EB_conc[k])
            }
    df = pd.DataFrame(pre_df) 
    
    df[Length] = abs((data['x2']-data['x1'])*pixel)
    data[Lifetime] = abs((data['y2']-data['y1'])*spf/60)
    
    df[Lifetime] = data[[Lifetime,'y2']].apply(lambda x: x[Lifetime] if x['y2'] < frames else np.NaN, axis=1)
    df[Lifetime] = data[[Lifetime,'y2']].apply(lambda x: x[Lifetime] if x[Lifetime] != 0 else np.NaN, axis=1)
    
    df[GrowthRate] = df[Length]/df[Lifetime]
    df[GrowthRate] = df[GrowthRate].apply(lambda x: x if x <= 5 else np.NAN )
    
    df[TimeToNucleate] = (data['y1']-start)*spf/60
    
    df[ShrinkageLength] = abs((data['x3']-data['x2'])*pixel)
    df[ShrinkageLifetime] = abs((data['y3']-data['y2'])*spf/60)
    df[ShrinkageLifetime] = df[ShrinkageLifetime].apply(lambda x: np.NAN if x == 0 else x)
    df[ShrinkageRate] = df[ShrinkageLength]/df[ShrinkageLifetime]
    
    df.loc[pd.isnull(df[GrowthRate]), 
           [Length,Lifetime,TimeToNucleate,ShrinkageLength,ShrinkageLifetime,ShrinkageRate]] = np.NaN
    
    df['Rescues'] = df[[Length,ShrinkageLength]].apply(lambda x: True if abs(x[Length]-x[ShrinkageLength]) > pixel*5 else False, axis=1)
    
    df['DCX Type'] = DCX_type[k]
    
    return df

In [357]:
#set path

path.parents[0],path

(WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/Mutant aggregation/tubulin'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/Mutant aggregation/tubulin/2023-05-16_DCX-EB'))

In [358]:
#set directory for new data file

newdirectory = str(path.parents[0])+'//Data_Sheets'
newmydir = Path(newdirectory)
newmydir.mkdir(exist_ok=True)

In [359]:
#Fill data frame with corrected ROI data

df = poly_dynamics(0)
for i in range(1,len(ch)):
    df0 = poly_dynamics(i)
    df = df.append(df0, ignore_index=True)

df.to_csv(path/(investigator+'_'+date +'.csv'), encoding='utf-8', index=False)
df.to_csv(newmydir/(investigator+'_'+date+'.csv'), encoding='utf-8', index=False)
df

,Date,Investigator,Ch,[Tubulin] $(\mu M)$,[DCX] $(n M)$,[EB] $(n M)$,Length $(\mu m)$,Lifetime $(min)$,Growth Rate $(\mu m / min)$,Time to Nucleate $(min)$,Shrink Length $(\mu m)$,Shrink Lifetime $(min)$,Shrink Rate $(\mu m / min)$,Rescues,DCX Type
0,2023-05-16,Brandi,1,8.0,0.0,0.0,0.749,1.213167,0.617392,2.849896,0.82925,0.041833,19.822709,False,None
1,2023-05-16,Brandi,1,8.0,0.0,0.0,1.783333,4.796888,0.371769,0.883729,1.961667,0.027888,70.339714,False,None
2,2023-05-16,Brandi,1,8.0,0.0,0.0,2.1935,3.137500,0.699124,7.577062,1.926,0.083667,23.01992,False,None
3,2023-05-16,Brandi,1,8.0,0.0,0.0,1.819,3.723167,0.488563,0.841896,1.926,0.209167,9.207968,False,None
4,2023-05-16,Brandi,1,8.0,0.0,0.0,5.992,10.207333,0.587029,0.507229,5.778,0.167333,34.52988,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2023-05-16,Brandi,1,8.0,0.0,0.0,6.634,10.207333,0.649925,2.013229,6.527,NaN,NaN,False,None
101,2023-05-16,Brandi,1,8.0,0.0,0.0,5.778,9.621667,0.600520,4.774229,5.778,0.083667,69.059761,False,None
102,2023-05-16,Brandi,1,8.0,0.0,0.0,7.597,12.382667,0.613519,1.176562,7.704,0.083667,92.079681,False,None
103,2023-05-16,Brandi,1,8.0,0.0,0.0,3.745,5.438333,0.688630,1.260229,3.745,0.083667,44.760956,False,None


In [26]:
df[['Date']].value_counts().reset_index(name='count')

,Date,count
0,,609


In [7]:
#open data frame directory for Joining files

askdirectory = filedialog.askdirectory() # show an "Open" dialog box and return the path to the selected file
path = Path(askdirectory)

In [8]:
#Find relevant files to concatenate

files = [x for x in path.iterdir() if x.is_file()]
files = [i for i in files if (str(i)[-10:] != 'results.csv') == True]
files = [i for i in files if (str(i)[-3:] == 'csv') == True]
files

[WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2022-12-19.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2022-12-21.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2023-01-02-2.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2023-01-02ch1.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2023-01-22ch1.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2023-01-29ch1.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2023-01-29ch2.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2023-02-11.csv'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Results_Files/DCX_EB_Dynamics/IRM/Master/Brandi_2023-02-12.csv'),
 WindowsPath('C:/Users/

In [9]:
#read files and concatenate into single joint file

data = pd.read_csv(files[0], encoding='utf-8')
for i in range(1,len(files)):
    d = pd.read_csv(files[i], encoding='utf-8')
    data = pd.concat([data,d])

In [10]:
#sort data frame 

data[Type] = data[Type].astype(category_Type)
data = data.sort_values(by=[Type, DCXconc])
#FIX CONCENTRATION
data[DCXconc] = data[DCXconc].apply(lambda x: round(x*2.27))
np.sort(data[DCXconc].unique())

array([  0, 136], dtype=int64)

In [11]:
#view data frame

data

,Date,Investigator,Ch,[Tubulin] $(\mu M)$,[DCX] $(n M)$,[EB] $(n M)$,Length $(\mu m)$,Lifetime $(min)$,Growth Rate $(\mu m / min)$,Time to Nucleate $(min)$,Shrink Length $(\mu m)$,Shrink Lifetime $(min)$,Shrink Rate $(\mu m / min)$,Rescues,DCX Type
0,2022-12-19,Brandi,1,6.0,0,0.0,0.6955,5.898500,0.117911,5.424389,1.070,0.251000,4.262948,False,None
1,2022-12-19,Brandi,1,6.0,0,0.0,0.8560,6.275000,0.136414,18.769222,1.070,0.251000,4.262948,False,None
2,2022-12-19,Brandi,1,6.0,0,0.0,0.8560,2.761000,0.310033,6.804889,0.856,0.167333,5.115538,False,None
3,2022-12-19,Brandi,1,6.0,0,0.0,1.0700,2.677333,0.399651,3.374556,1.070,0.167333,6.394422,False,None
4,2022-12-19,Brandi,1,6.0,0,0.0,0.7490,2.259000,0.331563,10.486222,0.856,0.083667,10.231076,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2023-04-24,Brandi,2,8.0,0,0.0,3.1030,7.613667,0.407557,3.137500,3.103,0.083667,37.087649,False,NaN
59,2023-04-24,Brandi,2,8.0,0,0.0,1.6050,3.597667,0.446122,11.838833,1.926,0.083667,23.019920,False,NaN
60,2023-04-24,Brandi,2,8.0,0,0.0,3.5310,7.111667,0.496508,15.603833,3.210,0.083667,38.366534,False,NaN
61,2023-04-24,Brandi,2,8.0,0,0.0,2.9960,6.944333,0.431431,5.898500,2.996,NaN,NaN,False,NaN


In [116]:
#save concatenated data frame as mastersheet 

data.to_csv(path/(date+'_MasterSheet.csv'), encoding='utf-8', index=False)

In [118]:
#Open concatenated data frame (mastersheet)

askmastersheet = filedialog.askopenfilename() # show an "Open" dialog box and return the path to the selected file
data = pd.read_csv(askmastersheet, encoding='utf-8')
data

,Date,Investigator,Ch,[Tubulin] $(\mu M)$,[DCX] $(n M)$,[EB] $(n M)$,Length $(\mu m)$,Lifetime $(min)$,Growth Rate $(\mu m / min)$,Time to Nucleate $(min)$,Shrink Length $(\mu m)$,Shrink Lifetime $(min)$,Shrink Rate $(\mu m / min)$,Rescues,DCX Type
0,2023-04-24,Brandi,1,8.0,0,0.0,7.169000,10.625667,0.674687,0.360812,7.383000,NaN,NaN,False,None
1,2023-04-24,Brandi,1,8.0,0,0.0,6.634000,10.040000,0.660757,8.811146,6.634000,0.251000,26.430279,False,None
2,2023-04-24,Brandi,1,8.0,0,0.0,10.272000,16.733333,0.613865,2.536146,10.272000,0.334667,30.693227,False,None
3,2023-04-24,Brandi,1,8.0,0,0.0,5.029000,9.287000,0.541510,11.655812,5.564000,0.167333,33.250996,True,None
4,2023-04-24,Brandi,1,8.0,0,0.0,4.708000,6.860667,0.686231,0.611812,4.387000,NaN,NaN,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,2023-04-28,Brandi,3,8.0,136,20.0,1.498000,1.547833,0.967804,6.386556,1.551500,0.083667,18.543825,False,P191R
399,2023-04-28,Brandi,3,8.0,136,20.0,1.391000,1.422333,0.977970,8.771056,1.337500,NaN,NaN,False,P191R
400,2023-04-28,Brandi,3,8.0,136,20.0,1.212667,1.143444,1.060539,8.547945,1.319667,0.083667,15.772908,False,P191R
401,2023-04-28,Brandi,3,8.0,136,20.0,1.498000,0.920333,1.627671,1.366556,1.658500,0.041833,39.645418,False,P191R


In [43]:
#Group data by mean, using (data parameter, as_index=False to reset index).calculation() 

GRmn='Mean'
dfmn=df.groupby(Type, as_index=False).mean()
dfmn.rename(columns={GrowthRate:GRmn}, inplace=True)
dfmn

,DCX Type,[Tubulin] $(\mu M)$,[DCX] $(n M)$,[EB] $(n M)$,Lifetime $(min)$,Mean,Shrink Lifetime $(min)$,Rescues
0,P191R,8.0,60.0,7.956989,7.872913,0.797853,0.187652,0.107527


In [45]:
#Group data by std dev.

GRstd='Std Dev.'
dfstd=df.groupby(Type, as_index=False).std()
dfstd.rename(columns={GrowthRate:GRstd}, inplace=True)
dfstd

,DCX Type,[Tubulin] $(\mu M)$,[DCX] $(n M)$,[EB] $(n M)$,Lifetime $(min)$,Std Dev.,Shrink Lifetime $(min)$,Rescues
0,P191R,0.0,0.0,9.842139,7.340392,0.237333,0.153098,0.311461


In [47]:
#merge mean and std dev data groups

dferr=pd.merge(dfmn,dfstd)
dferr

,DCX Type,[Tubulin] $(\mu M)$,[DCX] $(n M)$,[EB] $(n M)$,Lifetime $(min)$,Mean,Shrink Lifetime $(min)$,Rescues,Std Dev.


In [52]:
#define fit parameters

def gaussian(x, mu, sig):
    return (np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))) )/(sig*np.sqrt(2*np.pi))

def exponential(x, scale):
    return ((np.exp(-x/scale) )/(scale))

def gamma(x, shape, scale):
    return (np.power(x,shape-1)*np.exp(-x/ scale))/(sp.special.gamma(shape) * np.power(scale,shape))


In [53]:
#define bins

def make_hist(data, parameter, tubconc, dcxtype, dcxconc, ebconc) :
    
    selectdata = data[(data[tubulin]==tubconc)&(data[Type]==dcxtype)&(data[DCXconc]==dcxconc)&(data[EBconc]==ebconc)]
    
    if parameter == GrowthRate :
        maxbin = 1.5
        binsize = 0.05
    elif parameter == ShrinkageRate :
         maxbin = 15
         binsize = 0.1
    else:
        maxbin = 30
        binsize = 1
        
    n, bins, patches = plt.hist(selectdata[parameter], bins=np.arange(0, maxbin + binsize, binsize), density=True);
    plt.clf()
       
    return n, bins

In [54]:
#fit data 

def equation_fit(data, parameter):
    
    if (parameter ==  GrowthRate) | (parameter == ShrinkageRate) :
        equation = gaussian
    elif parameter == TimeToNucleate :
        equation = exponential
    elif parameter == Lifetime :
        equation = gamma
        
    results = pd.DataFrame(columns=[] , index=[])
    
    for tubconc in data[tubulin].unique():
        for dcxtype in data[Type].unique():
            for dcxconc in data[DCXconc].unique():
                for ebconc in data[EBconc].unique():

                    n, bins =  make_hist(data, parameter, tubconc, dcxtype, dcxconc,ebconc)      

                    if np.isnan(np.sum(n)) == True:
                        continue

                    if equation == gamma :
                        coeff, var_matrix = sp.optimize.curve_fit(equation,bins[:-1],n,[2,1])
                    else :
                        coeff, var_matrix = sp.optimize.curve_fit(equation,bins[:-1],n)

                    variance = np.diagonal(var_matrix) #Refer [3]
                    SE = np.sqrt(variance) #Refer [4]

                #======Making a data frame========
                    results0 = pd.DataFrame(columns=[] , index=[])
                    
                    for k in np.arange(0,len(coeff)):
                        
                        r0 = pd.DataFrame([[coeff[k],SE[k]],],columns=['Fit', 'SE'])
                        r0['Coefficient'] = k
                        results0 = pd.concat([results0, r0], sort=False)
                        
#                     for k in np.arange(0,len(coeff)):
#                         header = [np.array([parameter]),np.array(['Coefficient '+ str(k)])]
#                         r0 = pd.DataFrame([coeff[k],SE[k]], index=(['Value','SE']),columns= header)
#                         results0 = pd.concat([results0, r0], axis=1, sort=False)

                    results0[tubulin] = tubconc
                    results0[Type] = dcxtype
                    results0[DCXconc] = dcxconc
                    results0[EBconc] = ebconc
                    results0['Parameter'] = parameter

                    results = pd.concat([results, results0], sort=False)

    return results

In [55]:
#create directory for fit data

newmydir = path/(date+'_fitdata')
newmydir.mkdir(exist_ok=True)
newmydir

WindowsPath('C:/Users/Brouhard Lab/Data/Images/Data_Sheets/8uM tubulin/2023-04-28_fitdata')

In [56]:
#define fit variables

GrowthRateFit = equation_fit(data, GrowthRate);
TimeToNucleateFit = equation_fit(data, TimeToNucleate);
LifetimeFit = equation_fit(data, Lifetime);
ShrinkageRateFit = equation_fit(data, ShrinkageRate);

KeyError: 'DCX Type'

In [129]:
#define fit parameters

GrowthRateFit['Moment'] = GrowthRateFit['Coefficient'].apply(lambda x: 'Mean' if x == 0 else 'Std')
TimeToNucleateFit['Moment'] = 'Mean'
ShrinkageRateFit['Moment'] = ShrinkageRateFit['Coefficient'].apply(lambda x: 'Mean' if x == 0 else 'Std')

In [130]:
#visualize data frame

GrowthRateFit

,Fit,SE,Coefficient,[Tubulin] $(\mu M)$,DCX Type,[DCX] $(n M)$,[EB] $(n M)$,Parameter,Moment
0,0.353653,0.004863,0,6.0,None,0.0,0.0,Growth Rate $(\mu m / min)$,Mean
0,0.089388,0.003970,1,6.0,None,0.0,0.0,Growth Rate $(\mu m / min)$,Std
0,1.111208,0.020647,0,6.0,None,0.0,20.0,Growth Rate $(\mu m / min)$,Mean
0,0.207921,0.017165,1,6.0,None,0.0,20.0,Growth Rate $(\mu m / min)$,Std
0,0.381329,0.030550,0,6.0,WT,60.0,0.0,Growth Rate $(\mu m / min)$,Mean
0,0.169476,0.025077,1,6.0,WT,60.0,0.0,Growth Rate $(\mu m / min)$,Std
0,0.714622,0.024185,0,6.0,WT,60.0,20.0,Growth Rate $(\mu m / min)$,Mean
0,0.239216,0.019757,1,6.0,WT,60.0,20.0,Growth Rate $(\mu m / min)$,Std
0,0.403585,0.001373,0,6.0,R76S,60.0,0.0,Growth Rate $(\mu m / min)$,Mean
0,0.036117,0.000932,1,6.0,R76S,60.0,0.0,Growth Rate $(\mu m / min)$,Std


In [131]:
#build array for lifetimes

LifetimeCoeff0 = LifetimeFit[(LifetimeFit['Coefficient']==0)]['Fit'].values
LifetimeCoeff1 = LifetimeFit[(LifetimeFit['Coefficient']==1)]['Fit'].values
LifetimeSE0 = LifetimeFit[(LifetimeFit['Coefficient']==0)]['SE'].values
LifetimeSE1 = LifetimeFit[(LifetimeFit['Coefficient']==1)]['SE'].values
LifetimeMean = LifetimeCoeff0*LifetimeCoeff1
LifetimeSE = np.sqrt(np.square(LifetimeCoeff0*LifetimeSE1) + np.square(LifetimeCoeff1*LifetimeSE0))
LifetimeSE

array([0.64694597, 0.58110092, 6.14646184, 0.84571497, 2.4050492 ,
       2.17938587, 2.9673462 , 1.00100572, 6.70848803, 0.49726869,
       0.47477214, 8.18715562, 1.00506037, 3.04707913, 0.20009342,
       4.03853213, 0.42252535])

In [132]:
LTmeandf = LifetimeFit[(LifetimeFit['Coefficient']==0)]
LTmeandf['Fit'] = LifetimeMean
LTmeandf['SE'] = LifetimeSE
LTmeandf['Coefficient'] = np.NaN
LTmeandf['Moment'] = 'Mean'
LTmeandf

C:\Users\Brandi\AppData\Local\Temp\ipykernel_19628\1969975474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LTmeandf['Fit'] = LifetimeMean
C:\Users\Brandi\AppData\Local\Temp\ipykernel_19628\1969975474.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LTmeandf['SE'] = LifetimeSE
C:\Users\Brandi\AppData\Local\Temp\ipykernel_19628\1969975474.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,Fit,SE,Coefficient,[Tubulin] $(\mu M)$,DCX Type,[DCX] $(n M)$,[EB] $(n M)$,Parameter,Moment
0,4.708699,0.646946,NaN,6.0,None,0.0,0.0,Lifetime $(min)$,Mean
0,1.200566,0.581101,NaN,6.0,None,0.0,20.0,Lifetime $(min)$,Mean
0,14.935546,6.146462,NaN,6.0,WT,60.0,0.0,Lifetime $(min)$,Mean
0,1.094865,0.845715,NaN,6.0,WT,60.0,20.0,Lifetime $(min)$,Mean
0,7.609735,2.405049,NaN,6.0,R76S,60.0,0.0,Lifetime $(min)$,Mean
0,0.612170,2.179386,NaN,6.0,R76S,60.0,20.0,Lifetime $(min)$,Mean
0,8.275168,2.967346,NaN,6.0,P191R,60.0,0.0,Lifetime $(min)$,Mean
0,0.903116,1.001006,NaN,6.0,P191R,60.0,20.0,Lifetime $(min)$,Mean
0,12.507039,6.708488,NaN,10.0,None,0.0,0.0,Lifetime $(min)$,Mean
0,1.166154,0.497269,NaN,10.0,None,0.0,20.0,Lifetime $(min)$,Mean


In [133]:
#Fit results

ResultFit = pd.concat([GrowthRateFit, TimeToNucleateFit,LifetimeFit,LTmeandf,ShrinkageRateFit], axis=0, sort=False)
ResultFit.to_csv(newmydir/('ResultFit_'+date+'.csv'), encoding='utf-8', index=False)
ResultFit

,Fit,SE,Coefficient,[Tubulin] $(\mu M)$,DCX Type,[DCX] $(n M)$,[EB] $(n M)$,Parameter,Moment
0,0.353653,0.004863,0.0,6.0,None,0.0,0.0,Growth Rate $(\mu m / min)$,Mean
0,0.089388,0.003970,1.0,6.0,None,0.0,0.0,Growth Rate $(\mu m / min)$,Std
0,1.111208,0.020647,0.0,6.0,None,0.0,20.0,Growth Rate $(\mu m / min)$,Mean
0,0.207921,0.017165,1.0,6.0,None,0.0,20.0,Growth Rate $(\mu m / min)$,Std
0,0.381329,0.030550,0.0,6.0,WT,60.0,0.0,Growth Rate $(\mu m / min)$,Mean
...,...,...,...,...,...,...,...,...,...
0,0.582669,0.115932,1.0,10.0,R76S,60.0,20.0,Shrink Rate $(\mu m / min)$,Std
0,14.398055,0.618241,0.0,10.0,P191R,60.0,0.0,Shrink Rate $(\mu m / min)$,Mean
0,1.957969,0.431198,1.0,10.0,P191R,60.0,0.0,Shrink Rate $(\mu m / min)$,Std
0,10.012602,1.212951,0.0,10.0,P191R,60.0,20.0,Shrink Rate $(\mu m / min)$,Mean
